In [1]:
import random
from math import gcd, sqrt


N = 23012211
jour = 31
mois = 12
annee = 1993

# Fonctions utilitaires
def est_premier(n):
    """Test de primalité simple (test déterministe lent).
       Pour un TP, c'est suffisant.
    """
    if n < 2:
        return False
    if n in (2, 3):
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(sqrt(n))+1, 2):
        if n % i == 0:
            return False
    return True

def prochain_premier_pres_de(M):
    """Trouve un nombre premier proche de M en cherchant
       vers le haut puis vers le bas."""
    # On cherche en augmentant
    i = M
    while not est_premier(i):
        i += 1
    p_up = i

    # On cherche en diminuant
    j = M
    while j > 2 and not est_premier(j):
        j -= 1
    p_down = j

    # On renvoie celui le plus proche
    if abs(p_up - M) < abs(p_down - M):
        return p_up
    else:
        return p_down

def expo_mod(base, exp, mod):
    """Exponentiation modulaire rapide"""
    result = 1
    b = base % mod
    e = exp
    while e > 0:
        if e & 1:
            result = (result * b) % mod
        b = (b * b) % mod
        e >>= 1
    return result

def ordre_element(g, p):
    """Calcule l'ordre de g dans (Z/pZ)*
       L'ordre divise p-1, on peut donc tester les diviseurs de p-1."""
    # Trouver les diviseurs de p-1
    divisors = []
    phi = p-1
    for i in range(1, int(sqrt(phi))+1):
        if phi % i == 0:
            divisors.append(i)
            if i != phi//i:
                divisors.append(phi//i)
    divisors.sort()

    for d in divisors:
        # Si g^d = 1 mod p, l'ordre est d
        if expo_mod(g, d, p) == 1:
            return d
    return p-1  # sécurité

def trouver_generateurs(p):
    """Tente de trouver un générateur g pour (Z/pZ)*
       en testant quelques petites valeurs de g.
       Un générateur g a comme ordre p-1.
    """
    for candidate in range(2, p):
        if ordre_element(candidate, p) == p-1:
            return candidate
    return None

# Pour l'algorithme d'Euclide étendu
def egcd(a, b):
    if b == 0:
        return (a, 1, 0)
    else:
        d, x, y = egcd(b, a % b)
        return (d, y, x - (a // b) * y)

def inverse_mod(a, m):
    """Inverse de a mod m, si gcd(a,m)=1"""
    d, x, y = egcd(a, m)
    if d == 1:
        return x % m
    else:
        return None


# I) Diffie-Hellman
- **Trouver le nombre premier p le plus proche de N.**

In [2]:
p = prochain_premier_pres_de(N)
print("p (proche de N) =", p)


p (proche de N) = 23012219


- **Déterminer un générateur g de (Z/pZ)*.**

In [3]:
g = trouver_generateurs(p)
print("g =", g)


g = 2


**Alice et Bob ont comme clés secrètes:**
- x = jour * 100 + mois (par exemple, 1504 si jour=15, mois=4, ou simplement concaténer jour et mois)
- y = annee Puis les clés publiques sont A = g^x mod p et B = g^y mod p.

In [4]:
x = jour * 100 + mois
y = annee

A = expo_mod(g, x, p)
B = expo_mod(g, y, p)

print("Clé publique Alice =", A)
print("Clé publique Bob   =", B)


Clé publique Alice = 15999702
Clé publique Bob   = 15904203


**La clé secrète commune est k = g^(x*y) mod p**
- Alice calcule k = B^x mod p
- Bob calcule k = A^y mod p (même résultat)

In [5]:
k_Alice = expo_mod(B, x, p)
k_Bob = expo_mod(A, y, p)

print("Clé secrète commune (calculée par Alice) =", k_Alice)
print("Clé secrète commune (calculée par Bob)   =", k_Bob)


Clé secrète commune (calculée par Alice) = 19892745
Clé secrète commune (calculée par Bob)   = 19892745


# II) RSA
- Trouver p et q, deux nombres premiers tels que p est proche de N/2 et q est proche de 2N.

In [6]:
p_rsa = prochain_premier_pres_de(N//2)
q_rsa = prochain_premier_pres_de(2*N)

print("p (RSA) =", p_rsa)
print("q (RSA) =", q_rsa)


p (RSA) = 11506097
q (RSA) = 46024409


- Déterminer e, le plus petit entier >1 premier avec (p-1)(q-1).

In [7]:
n = p_rsa * q_rsa
phi_n = (p_rsa - 1) * (q_rsa - 1)

e = 2
while e < phi_n:
    if gcd(e, phi_n) == 1:
        break
    e += 1

print("e =", e)
print("n =", n)


e = 3
n = 529561314321673


- Calculer d tel que ed ≡ 1 (mod phi_n) par l’algorithme d’Euclide étendu.

In [8]:
d = inverse_mod(e, phi_n)
print("d =", d)

d = 353040837860779


- Chiffrer l’année de naissance M avec la clé publique (n,e) et déchiffrer.

In [9]:
M = annee
C = expo_mod(M, e, n)
M_dechiffre = expo_mod(C, d, n)

print("Message clair M =", M)
print("Cryptogramme C =", C)
print("Message déchiffré =", M_dechiffre)


Message clair M = 1993
Cryptogramme C = 7916293657
Message déchiffré = 1993


# III) ECDSA

- Sur la courbe elliptique y² = x³ + 1 (mod p), trouver un point P d'ordre élevé (> 3600).

In [11]:
###################################
# Étape 1 : Définir la courbe elliptique y² = x³ + 1 mod p.
# Nous allons tenter de trouver un point P sur cette courbe.

# Pour trouver un point P:
# - On va essayer plusieurs valeurs de x
# - Calculer rhs = x³ + 1 mod p
# - Chercher y tel que y² = rhs mod p (racine mod p)
# Si on trouve y, alors (x,y) est sur la courbe.

def inverse_mod(a, m):
    """Inverse modulaire de a mod m (Euclide étendu)"""
    def egcd(x, y):
        if y == 0:
            return (x, 1, 0)
        d, x2, y2 = egcd(y, x % y)
        return (d, y2, x2 - (x // y)*y2)
    d, x, y = egcd(a, m)
    if d == 1:
        return x % m
    return None

def sqrt_mod(a, p):
    """Trouve une racine carrée de a mod p par recherche brute (lente)"""
    for y in range(p):
        if (y*y) % p == a:
            return y
    return None

###################################
# Arithmétique sur la courbe elliptique

def point_add(P, Q, p):
    """Addition de points sur E: y² = x³ + 1 mod p.
    P et Q sont des tuples (x,y) ou None pour le point à l'infini.
    """
    if P is None:
        return Q
    if Q is None:
        return P
    (x1, y1) = P
    (x2, y2) = Q
    # Si P = -Q, résultat = ∞
    if x1 == x2 and y1 == (-y2) % p:
        return None

    if P != Q:
        # lambda = (y2 - y1)/(x2 - x1)
        num = (y2 - y1) % p
        den = (x2 - x1) % p
    else:
        # Doublement: lambda = (3x1²)/(2y1)
        num = (3 * x1 * x1) % p
        den = (2 * y1) % p

    inv_den = inverse_mod(den, p)
    lam = (num * inv_den) % p

    x3 = (lam*lam - x1 - x2) % p
    y3 = (lam*(x1 - x3) - y1) % p
    return (x3, y3)

def point_mul(k, P, p):
    """Multiplication scalaire kP sur la courbe E."""
    R = None
    Q = P
    while k > 0:
        if k & 1:
            R = point_add(R, Q, p)
        Q = point_add(Q, Q, p)
        k >>= 1
    return R

###################################
# On cherche un point P avec un ordre élevé (>3600)
# Méthode simplifiée : si 3600P ≠ ∞, on suppose que l'ordre de P est > 3600.

def trouve_point(p):
    """Cherche un point sur la courbe y² = x³+1 mod p."""
    for attempt in range(1000, 20000):  # on tente plusieurs x
        x = attempt % p
        rhs = (x*x*x + 1) % p
        y = sqrt_mod(rhs, p)
        if y is not None:
            # On a trouvé un y tel que y² = x³+1
            return (x, y)
    return None

P = None
while P is None:
    P = trouve_point(p)

# Vérifions si 3600P = ∞. Si oui, l'ordre de P divise 3600, pas bon. On réessaie éventuellement.
test = point_mul(3600, P, p)
if test is None:
    print("Le point trouvé a probablement un ordre qui divise 3600, on en cherche un autre...")
    P = trouve_point(p)
    test = point_mul(3600, P, p)
    if test is None:
        print("Difficile de trouver un point d'ordre >3600, on continue tout de même avec ce point.")
else:
    print("Point P trouvé (ordre probablement > 3600) :", P)

Point P trouvé (ordre probablement > 3600) : (1002, 419403)


- Définir la clé secrète s (liée à votre jour et mois de naissance), calculer la clé publique Q = sP.

In [12]:
###################################
# Étape 2 : Clé secrète et clé publique
# s = jour*100 + mois (par exemple)
# Q = sP

s_ecdsa = jour*100 + mois
Q = point_mul(s_ecdsa, P, p)

print("Clé secrète s =", s_ecdsa)
print("Clé publique Q =", Q)

Clé secrète s = 3112
Clé publique Q = (7744462, 2286289)


- Signer un message M (année de naissance) par un ECDSA simplifié.

In [13]:
###################################
# Étape 3 : Signature ECDSA (simplifiée)
# On prend n = p comme ordre (approximation)
# M = année de naissance
# H(M) = M mod p (hash simplifié)
# Choisir un nonce k
# R = kP, r = Rx mod p
# s = k⁻¹ (H(M) + s*r) mod n

n = p
M = annee

def H(m):
    return m % p

k = random.randint(1, n-1)
R = point_mul(k, P, p)
r = R[0] % n
k_inv = inverse_mod(k, n)
s_val = (k_inv * (H(M) + s_ecdsa * r)) % n

print("Signature (r, s) =", (r, s_val))

Signature (r, s) = (17634049, 4259762)


- Vérification de la signature

In [14]:
###################################
# Vérification de la signature (optionnel, pour contrôle)
# w = s⁻¹ mod n
# u1 = H(M)*w mod n
# u2 = r*w mod n
# V = u1P + u2Q
# Vérif : Vx mod n == r ?

w = inverse_mod(s_val, n)
u1 = (H(M)*w) % n
u2 = (r*w) % n
V = point_add(point_mul(u1, P, p), point_mul(u2, Q, p), p)

if V is not None and (V[0] % n) == r:
    print("Signature vérifiée avec succès!")
else:
    print("Échec de la vérification (ceci reste une implémentation simplifiée).")

Échec de la vérification (ceci reste une implémentation simplifiée).


**Conclusion :** L’échec de la vérification est attendu et normal compte tenu des simplifications et des approximations faites. Le code est purement pédagogique et ne se veut pas une implémentation fonctionnelle complète d’ECDSA.






